In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

In [3]:
base_dir=os.path.join(os.path.dirname('/content/gdrive/MyDrive/PetImages'),'Dataset')

In [4]:
IMAGE_SIZE=200
BATCH_SIZE=64
datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [5]:
train_generator=datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE,IMAGE_SIZE),batch_size=BATCH_SIZE,subset='training')

Found 232 images belonging to 6 classes.


In [6]:
val_generator=datagen.flow_from_directory(base_dir,target_size=(IMAGE_SIZE,IMAGE_SIZE),batch_size=BATCH_SIZE,subset='validation')

Found 58 images belonging to 6 classes.


In [7]:
labels='\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt','w') as file:
  file.write(labels)

In [9]:
base_model=tf.keras.applications.MobileNetV2(input_shape=(200,200,3),include_top=False,weights='imagenet')
base_model.trainable=False

In [10]:
model=tf.keras.Sequential([
                           base_model,
                           tf.keras.layers.Conv2D(32,3,activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.GlobalAveragePooling2D(),
                           tf.keras.layers.Dense(6,activation='softmax')
])

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 2,626,854
Trainable params: 368,870
Non-trainable params: 2,257,984
_________________________________________________________________


In [13]:
history=model.fit_generator(train_generator,epochs=10,steps_per_epoch=50,validation_data=val_generator,validation_steps=50)
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
50/50 [==============================] - 81s 1s/step - loss: 2.8795 - accuracy: 0.3750 - val_loss: 0.5899 - val_accuracy: 0.7931


In [14]:
base_model.trainable=True

In [15]:
fine_tune_at=100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False

In [16]:
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
conv2d (Conv2D)              (None, 5, 5, 32)          368672    
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 2,626,854
Trainable params: 2,230,310
Non-trainable params: 396,544
_________________________________________________________________


In [18]:
hist_fine=model.fit_generator(train_generator,epochs=5,steps_per_epoch=50,validation_data=val_generator,validation_steps=50)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
50/50 [==============================] - 24s 349ms/step - loss: 1.0905 - accuracy: 0.5905 - val_loss: 2.0668 - val_accuracy: 0.6724


In [19]:
save_model_dir='save/fine_tuning'
tf.saved_model.save(model,save_model_dir)
convert=tf.lite.TFLiteConverter.from_saved_model(save_model_dir)
tflite_model=convert.convert()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.


INFO:tensorflow:Assets written to: save/fine_tuning/assets


INFO:tensorflow:Assets written to: save/fine_tuning/assets


In [20]:
with open('model.tflite','wb') as file:
  file.write(tflite_model)